In [1]:
import argparse
from pathlib import Path
from fastai.imagito.utils import *
from fastai.vision.models.xresnet2 import xresnet50_2
import torch
import math
from operator import itemgetter

In [16]:
def load_model(model_dir, size, woof, bs, sample):
#     model_params = pickle_load(model_dir + '/params.pkl')
#     if model_params['size'] != size:
#         print('Warning: Model was trained on image size=%d but is being used to predict for size=%d'
#               % (model_params['size'], size))

    m = xresnet50_2
    #print(model_params)
    data = get_data(size, woof, bs, sample, shuffle_train=False, flip_lr_p=0.0)
    l = Learner(data, m(c_out=10), path=model_dir)
    l.load('woof_classif')
    return l, data

In [ ]:
!mv woof_classif.pth

In [4]:
mkdir models

In [5]:
mv woof_classif.pth models

In [15]:
ls models

woof_classif.pth


In [19]:
model, data = load_model('.', 128, True, 256, 1.)

In [20]:
predictions_t, targets_t, loss_t = model.get_preds(ds_type=DatasetType.Train, with_loss=True)

In [21]:
to_arr = lambda x: x.numpy()

In [27]:
paths = list(map(lambda x: x.name,data.items))

['n02115641_44296.JPEG',
 'n02115641_7648.JPEG',
 'n02088364_1449.JPEG',
 'n02099601_8874.JPEG',
 'n02089973_4875.JPEG',
 'n02099601_447.JPEG',
 'n02115641_28502.JPEG',
 'n02093754_4460.JPEG',
 'n02093754_6370.JPEG',
 'n02086240_3672.JPEG',
 'n02093754_4384.JPEG',
 'n02111889_15388.JPEG',
 'n02088364_2661.JPEG',
 'n02111889_3439.JPEG',
 'n02115641_925.JPEG',
 'n02093754_5198.JPEG',
 'n02093754_1310.JPEG',
 'n02096294_6439.JPEG',
 'n02093754_696.JPEG',
 'n02099601_729.JPEG',
 'n02093754_5658.JPEG',
 'n02115641_40643.JPEG',
 'n02087394_5692.JPEG',
 'n02099601_11459.JPEG',
 'n02088364_4219.JPEG',
 'n02087394_11890.JPEG',
 'n02099601_367.JPEG',
 'n02105641_7918.JPEG',
 'n02088364_15440.JPEG',
 'n02087394_17317.JPEG',
 'n02087394_9704.JPEG',
 'n02089973_3577.JPEG',
 'n02115641_5663.JPEG',
 'n02087394_8863.JPEG',
 'n02111889_16768.JPEG',
 'n02105641_4355.JPEG',
 'n02087394_2584.JPEG',
 'n02087394_15985.JPEG',
 'n02086240_9824.JPEG',
 'n02086240_1117.JPEG',
 'n02087394_7544.JPEG',
 'n02089973

In [33]:
df = pd.DataFrame(to_arr(loss_t), columns=['loss'])
df.index = paths

In [43]:
df = df.rename_axis('paths').sort_values('loss', ascending=False).reset_index()

In [44]:
df.to_msgpack('/home/paperspace/fastai-fork/fastai/imagito/pred_df_woof.mp')

In [45]:
from fastai.imagito.train_imagito import main

In [38]:
%debug

> /home/paperspace/anaconda3/envs/fastaiv1/lib/python3.6/site-packages/pandas/io/packers.py(226)read_msgpack()
    224         return read(path_or_buf)
    225 
--> 226     raise ValueError('path_or_buf needs to be a string file path or file-like')
    227 
    228 

ipdb> u
> /home/paperspace/fastai-fork/fastai/imagito/sample_hardness.py(139)make_hardness_filter_func()
    137     if (hardness_lower_bound == 0) and (hardness_upper_bound==1.):
    138         return return_true
--> 139     pred_df = pd.read_msgpack(path).sort_values('loss', ascending=False)
    140     start_idx, end_idx = int(IMAGENETTE_SIZE * hardness_lower_bound), int(
    141         IMAGENETTE_SIZE * hardness_upper_bound) - 1

ipdb> path
PosixPath('/home/paperspace/fastai-fork/fastai/imagito/pred_df_woof.mp')
ipdb> exit


In [32]:
df

,0
n02115641_44296.JPEG,0.112671
n02115641_7648.JPEG,0.235310
n02088364_1449.JPEG,0.863584
n02099601_8874.JPEG,0.386320
n02089973_4875.JPEG,1.649868
n02099601_447.JPEG,0.195452
n02115641_28502.JPEG,2.505424
n02093754_4460.JPEG,0.003552
n02093754_6370.JPEG,0.300552
n02086240_3672.JPEG,0.000826
